# Joins 

Joins an essential part of nearly all Spark workloads. Here is how to use them (the basic internals).

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder
    .appName("blabla")
    .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://07ff17ebfed3:4040
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1597088989115)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@164c182f


In [2]:
val person = Seq(
    (0, "Bill Chambers", 0, Seq(100)),
    (1, "Matei Zaharia", 1, Seq(500, 250, 100)),
    (2, "Michael Armbrust", 1, Seq(250, 100)))
    .toDF("id_person", "name", "graduate_program", "spark_status")

val graduateProgram = Seq(
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley"))
    .toDF("id_program", "degree", "department", "school")

val sparkStatus = Seq(
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor"))
    .toDF("id", "status")

person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

person.show()
graduateProgram.show()
sparkStatus.show()

+---------+----------------+----------------+---------------+
|id_person|            name|graduate_program|   spark_status|
+---------+----------------+----------------+---------------+
|        0|   Bill Chambers|               0|          [100]|
|        1|   Matei Zaharia|               1|[500, 250, 100]|
|        2|Michael Armbrust|               1|     [250, 100]|
+---------+----------------+----------------+---------------+

+----------+-------+--------------------+-----------+
|id_program| degree|          department|     school|
+----------+-------+--------------------+-----------+
|         0|Masters|School of Informa...|UC Berkeley|
|         2|Masters|                EECS|UC Berkeley|
|         1|  Ph.D.|                EECS|UC Berkeley|
+----------+-------+--------------------+-----------+

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+



person: org.apache.spark.sql.DataFrame = [id_person: int, name: string ... 2 more fields]
graduateProgram: org.apache.spark.sql.DataFrame = [id_program: int, degree: string ... 2 more fields]
sparkStatus: org.apache.spark.sql.DataFrame = [id: int, status: string]


# Inner joins

In [3]:
var joinExpr = person.col("graduate_program") === graduateProgram.col("id_program")
person.join(graduateProgram, joinExpr, "inner").show() // inner can be ommitted

spark.sql("""
SELECT *
FROM person INNER JOIN graduateProgram 
ON person.graduate_program = graduateProgram.id_program
""").show() // INNER can be ommitted

+---------+----------------+----------------+---------------+----------+-------+--------------------+-----------+
|id_person|            name|graduate_program|   spark_status|id_program| degree|          department|     school|
+---------+----------------+----------------+---------------+----------+-------+--------------------+-----------+
|        0|   Bill Chambers|               0|          [100]|         0|Masters|School of Informa...|UC Berkeley|
|        2|Michael Armbrust|               1|     [250, 100]|         1|  Ph.D.|                EECS|UC Berkeley|
|        1|   Matei Zaharia|               1|[500, 250, 100]|         1|  Ph.D.|                EECS|UC Berkeley|
+---------+----------------+----------------+---------------+----------+-------+--------------------+-----------+



joinExpr: org.apache.spark.sql.Column = (graduate_program = id_program)


# Outer Joins

In [6]:
person.join(graduateProgram, joinExpr, "outer").show()

spark.sql("""
SELECT *
FROM person FULL OUTER JOIN graduateProgram 
ON person.graduate_program = graduateProgram.id_program
""").show()

+---------+----------------+----------------+---------------+----------+-------+--------------------+-----------+
|id_person|            name|graduate_program|   spark_status|id_program| degree|          department|     school|
+---------+----------------+----------------+---------------+----------+-------+--------------------+-----------+
|        1|   Matei Zaharia|               1|[500, 250, 100]|         1|  Ph.D.|                EECS|UC Berkeley|
|        2|Michael Armbrust|               1|     [250, 100]|         1|  Ph.D.|                EECS|UC Berkeley|
|     null|            null|            null|           null|         2|Masters|                EECS|UC Berkeley|
|        0|   Bill Chambers|               0|          [100]|         0|Masters|School of Informa...|UC Berkeley|
+---------+----------------+----------------+---------------+----------+-------+--------------------+-----------+



# Left Outer Joins

In [9]:
graduateProgram.join(person, joinExpr, "left_outer").show()

spark.sql("""
SELECT *
FROM graduateProgram LEFT OUTER JOIN person
ON person.graduate_program = graduateProgram.id_program
""").show()

+----------+-------+--------------------+-----------+---------+----------------+----------------+---------------+
|id_program| degree|          department|     school|id_person|            name|graduate_program|   spark_status|
+----------+-------+--------------------+-----------+---------+----------------+----------------+---------------+
|         0|Masters|School of Informa...|UC Berkeley|        0|   Bill Chambers|               0|          [100]|
|         2|Masters|                EECS|UC Berkeley|     null|            null|            null|           null|
|         1|  Ph.D.|                EECS|UC Berkeley|        2|Michael Armbrust|               1|     [250, 100]|
|         1|  Ph.D.|                EECS|UC Berkeley|        1|   Matei Zaharia|               1|[500, 250, 100]|
+----------+-------+--------------------+-----------+---------+----------------+----------------+---------------+



# Right Outer Joins

In [11]:
person.join(graduateProgram, joinExpr, "right_outer").show()

spark.sql("""
SELECT *
FROM person RIGHT OUTER JOIN graduateProgram
ON person.graduate_program = graduateProgram.id_program
""").show()

+---------+----------------+----------------+---------------+----------+-------+--------------------+-----------+
|id_person|            name|graduate_program|   spark_status|id_program| degree|          department|     school|
+---------+----------------+----------------+---------------+----------+-------+--------------------+-----------+
|        0|   Bill Chambers|               0|          [100]|         0|Masters|School of Informa...|UC Berkeley|
|     null|            null|            null|           null|         2|Masters|                EECS|UC Berkeley|
|        2|Michael Armbrust|               1|     [250, 100]|         1|  Ph.D.|                EECS|UC Berkeley|
|        1|   Matei Zaharia|               1|[500, 250, 100]|         1|  Ph.D.|                EECS|UC Berkeley|
+---------+----------------+----------------+---------------+----------+-------+--------------------+-----------+



# Left Semi Joins

In [32]:
graduateProgram.join(person, joinExpr, "left_semi").show()

spark.sql("""
SELECT *
FROM graduateProgram LEFT SEMI JOIN person 
ON person.graduate_program = graduateProgram.id_program
""").show()

+----------+-------+--------------------+-----------+
|id_program| degree|          department|     school|
+----------+-------+--------------------+-----------+
|         0|Masters|School of Informa...|UC Berkeley|
|         1|  Ph.D.|                EECS|UC Berkeley|
+----------+-------+--------------------+-----------+



# Left Anti Joins

The opposite of let semi joins

In [34]:
graduateProgram.join(person, joinExpr, "left_anti").show()

spark.sql("""
SELECT *
FROM graduateProgram LEFT ANTI JOIN person 
ON person.graduate_program = graduateProgram.id_program
""").show()

+----------+-------+----------+-----------+
|id_program| degree|department|     school|
+----------+-------+----------+-----------+
|         2|Masters|      EECS|UC Berkeley|
+----------+-------+----------+-----------+



# Natural Joins

Natural joints make implicit guesses at the columns on which you would like to join. Implicit are always dangerous !

In [36]:
spark.sql("""
SELECT * FROM graduateProgram NATURAL JOIN person
""").show()

+----------+-------+--------------------+-----------+---------+----------------+----------------+---------------+
|id_program| degree|          department|     school|id_person|            name|graduate_program|   spark_status|
+----------+-------+--------------------+-----------+---------+----------------+----------------+---------------+
|         0|Masters|School of Informa...|UC Berkeley|        0|   Bill Chambers|               0|          [100]|
|         0|Masters|School of Informa...|UC Berkeley|        1|   Matei Zaharia|               1|[500, 250, 100]|
|         0|Masters|School of Informa...|UC Berkeley|        2|Michael Armbrust|               1|     [250, 100]|
|         2|Masters|                EECS|UC Berkeley|        0|   Bill Chambers|               0|          [100]|
|         2|Masters|                EECS|UC Berkeley|        1|   Matei Zaharia|               1|[500, 250, 100]|
|         2|Masters|                EECS|UC Berkeley|        2|Michael Armbrust|        

# Cross (Cartesian) Joins 

They are like inner joins taht do not specify a predicate. Cross joins will join every single row in the left DF to every single row in the right DF. If you've 1,000 rows in each DF, it'll result in 1,000 x 1,000 rows ! So you've to be absolutely sure this is what you want... and you've to explicitly specify it :

In [17]:
graduateProgram.join(person, joinExpr, "cross").show()

spark.sql("""
SELECT *
FROM graduateProgram CROSS JOIN person 
ON person.graduate_program = graduateProgram.id_program
""").show()

+----------+-------+--------------------+-----------+---------+----------------+----------------+---------------+
|id_program| degree|          department|     school|id_person|            name|graduate_program|   spark_status|
+----------+-------+--------------------+-----------+---------+----------------+----------------+---------------+
|         0|Masters|School of Informa...|UC Berkeley|        0|   Bill Chambers|               0|          [100]|
|         1|  Ph.D.|                EECS|UC Berkeley|        2|Michael Armbrust|               1|     [250, 100]|
|         1|  Ph.D.|                EECS|UC Berkeley|        1|   Matei Zaharia|               1|[500, 250, 100]|
+----------+-------+--------------------+-----------+---------+----------------+----------------+---------------+



If you truly intend to have a cross-join, you can call that out explicitly:

In [39]:
graduateProgram.crossJoin(person).show()

spark.sql("""
SELECT * FROM graduateProgram CROSS JOIN person
""").show()

+----------+-------+--------------------+-----------+---------+----------------+----------------+---------------+
|id_program| degree|          department|     school|id_person|            name|graduate_program|   spark_status|
+----------+-------+--------------------+-----------+---------+----------------+----------------+---------------+
|         0|Masters|School of Informa...|UC Berkeley|        0|   Bill Chambers|               0|          [100]|
|         0|Masters|School of Informa...|UC Berkeley|        1|   Matei Zaharia|               1|[500, 250, 100]|
|         0|Masters|School of Informa...|UC Berkeley|        2|Michael Armbrust|               1|     [250, 100]|
|         2|Masters|                EECS|UC Berkeley|        0|   Bill Chambers|               0|          [100]|
|         2|Masters|                EECS|UC Berkeley|        1|   Matei Zaharia|               1|[500, 250, 100]|
|         2|Masters|                EECS|UC Berkeley|        2|Michael Armbrust|        